In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# LCEL for chain
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")

model = ChatOpenAI(
    model="gpt-4o-mini"
)

# StrOutputParser() ai 의 응답을 내용만 봅아서 볼 수 있음. 원래는 메타 데이터도 포함됨.
chain = prompt | model | StrOutputParser()

print(f"[답변]: {chain.invoke({'topic': 'ice cream'})}")



[답변]: Why did the ice cream cone become a detective? 

Because it always wanted to solve the case of the missing sprinkles!


In [7]:
# chain 의 stream 기능 써보기
model = ChatOpenAI(
    model="gpt-4o-mini"
)

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")

chain = prompt | model

for s in chain.stream({"topic": "ice cream"}):
    print(s.content, end="", flush=True)

Why did the ice cream truck break down?

Because it had a rocky road!


Stream 사용 이유
* 빠른 응답 제공: 전체 응답이 생성될 때까지 기다릴 필요 없이, 한 토큰씩 받아볼 수 있음.
* 실시간 처리 가능: UI에서 점진적으로 텍스트를 표시하거나, 스트림 데이터를 다른 곳에서 활용할 수 있음.
* 메모리 효율성: 한 번에 모든 데이터를 로드하지 않기 때문에 메모리 부담이 줄어듦.

In [14]:
# 프롬포트는 이렇게 해야된대
from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        # SystemMessage : ai 의 역할과 행동 지침을 설정.
        SystemMessage(
            content=(
                "너는 소설 전문가 AI야. 사용자가 원하는 장르의 영화를 리스트 형태로 추천해줘."
                'ex(Query: 현대 문학 소설 5개 추천해줘 / 답변: ["aa", "ee", "ee2"]'
                # 프롬포트를 아래 처럼 명확히 전달하면, 원하는 대로 나옴. Or model 의 temperature 을 내리는 것도 방법
                # "너는 소설 전문가 AI야. 사용자가 원하는 장르의 소설을 리스트 형태(예: [\"aa\", \"ee\", \"ee2\"])로만 추천해줘. "
                # "어떠한 설명이나 부연 없이 오직 리스트 형태로만 답변해."
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),  # HumanMessagePromptTemplate 사용자의 입력을 받기 위한 템플릿
    ]
)

model = ChatOpenAI(model="gpt-4o-mini") # temperature=0.2
chain = chat_template | model
chain.invoke("사회").content

'["1984 - 조지 오웰", "브레이킹 다운 - 마거릿 애트우드", "노인과 바다 - 어니스트 헤밍웨이", "안나 카레니나 - 레프 톨스토이", "소설가 구보 씨의 일일 - 박태원"]'

In [15]:
# 부분적으로 프롬포트에 미리 넣어두고 싶은 부분이 있다면?
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import CommaSeparatedListOutputParser

# parser 선언
output_parser = CommaSeparatedListOutputParser()

# csv 파서 작동을 위한 형식 지정 프롬프트 로드
format_instructions = output_parser.get_format_instructions()

# 프롬포트 템플릿의 partial_variables 에 csv 형식 지정 프롬포트 주입
prompt = PromptTemplate(
    template="List {subject}. {format_instructions}", # "List 한국 현대 문학. 지정한 포맷 형식으로 답변해 주세요."
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

model = ChatOpenAI(model="gpt-4o-mini")
chain = prompt | model | output_parser
chain.invoke({"subject":"한국 현대 문학"})



['김영하', '한강', '이민진', '박완서', '정유정', '최은영', '윤고은', '공지영', '김애란', '백영미']

In [36]:
# jsonOutPutParser 이용해보기
from pydantic import Field
from langchain_core.output_parsers import JsonOutputParser
from openai import BaseModel

class Country(BaseModel):
    continent: str = Field(description="사용자가 물어본 나라가 속한 대륙")
    population: str = Field(description="사용자가 물어본 나라의 인구(int 형식)")


# JsonOutputParser 생성할 때 format_instructions 로 class 넘겨
parser = JsonOutputParser(pydantic_object=Country)

prompt = PromptTemplate(
    template="Answer the user query. in korean. {format_instructions} {query}",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

country_query = "태국은 어떤 나라야?"
chain.invoke({"query": country_query})


{'continent': '아시아', 'population': '70000000'}

In [44]:
from langchain_core.runnables import RunnablePassthrough

# RunnablePassthrough 입력 그대로 전달
# 다음 한글 문장을 프랑스어로 번역해줘 윤지는 아무래도 짱이다.
# French sentence: (print from here)
prompt = ChatPromptTemplate.from_template(
    """다음 한글 문장을 프랑스어로 번역해줘 {sentence}
        French sentence: (print from here)"""
)

model = ChatOpenAI(model="gpt-4o-mini")
parser = StrOutputParser()

runnable_chain = {"sentence": RunnablePassthrough()} | prompt | model | parser
runnable_chain.invoke({"sentence": "윤지는 아무래도 짱이다."})

'윤지는 아무래도 짱이다.  \nFrench sentence: Yoonji est vraiment géniale.'

In [46]:
def add_smile(x):
    return x + " :)"

In [27]:
from langchain_core.runnables import RunnableLambda

 = RunnableLambda()

In [35]:
prompt_str = "{topic} 의 역사에 대해 세 문장으로 설명해주세요."
prompt = ChatPromptTemplate.from_template(prompt_str)

model = ChatOpenAI(model="gpt-4o-mini")

output_parser = StrOutputParser()

chain = prompt | model | output_parser | 
chain.invoke({"topic": "한국"})

'한국의 역사는 고조선(기원전 2333년)으로 시작되어 삼국 시대(고구려, 백제, 신라)를 거쳐 통일 신라와 고려 시대를 지나 조선 왕조에 이르렀습니다. 20세기 초 일본의 식민지 지배를 받았으나, 1945년 해방된 후 1948년 대한민국과 조선민주주의인민공화국으로 분단되었습니다. 이후 한국전쟁(1950-1953)을 겪고 경제 성장을 이루며 현재는 민주주의 국가로 자리잡고 있습니다.:)'

In [49]:
from langchain_core.runnables import RunnableParallel

# RunnableParallel 여러 작업 동시 병렬로 진행
# LLM 호출, API 요청, 전처리 등을 자유롭게 결합.
runnable_parallel = RunnableParallel(
    passed = RunnablePassthrough(),
    modified = add_smile
)

runnable_parallel.invoke("하이하이")

{'passed': '하이하이', 'modified': '하이하이 :)'}

In [50]:
# RunnableParallel 예제 2
from langchain_core.runnables import RunnableParallel
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# prompt
description_prompt = ChatPromptTemplate.from_template("{topic} 이 무엇을 뜻하는 약자입니까?")
celeb_prompt = ChatPromptTemplate.from_template("{topic} 와 관련된 유명인사는 누구입니까?")
# model
model = ChatOpenAI(model="gpt-4o-mini", max_tokens=128, temperature=0)
# output_parser
output_parser = StrOutputParser()

description_chain = description_prompt | model | output_parser
celeb_chain = celeb_prompt | model | output_parser

map_chain = RunnableParallel(description = description_chain, celeb = celeb_chain)
invoke= map_chain.invoke({"topic": "AI"})
invoke


{'description': 'AI는 "Artificial Intelligence"의 약자로, 한국어로는 "인공지능"이라고 합니다. 인공지능은 컴퓨터 시스템이나 기계가 인간의 지능을 모방하여 학습, 추론, 문제 해결 등의 작업을 수행할 수 있도록 하는 기술을 의미합니다.',
 'celeb': 'AI와 관련된 유명인사들은 여러 분야에서 활동하고 있으며, 그 중 일부는 다음과 같습니다:\n\n1. **엘론 머스크 (Elon Musk)** - 테슬라와 스페이스X의 CEO로, AI의 발전에 대한 경고와 함께 OpenAI의 공동 창립자이기도 합니다.\n\n2. **제프리 힌튼 (Geoffrey Hinton)** - 딥러닝의 아버지로 불리며, 인공지능 분야에서 중요한 연구를 해온 인물입니다.\n\n3. **얀 르쿤 (Yann LeCun)**'}